In [1]:
#| default_exp requirementsBot
%pip install -q langchain
%pip install -q langchain_community
%pip install -qU langchain-openai
%pip install -q pydantic

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [44]:
#| export
from langchain_core.messages import HumanMessage

from langchain_openai import ChatOpenAI

model = ChatOpenAI()


In [72]:
#| export 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

captureRequirementsPrompt = """
You are a requirements recorder, helping a job searcher, by recording the requirements they have in the following areas: location, tech_stack, employement_type, and salary. Listen to the user's message, and return a json object with two keys: 'new_requirements', and 'remove_requirements'. The first one listing new requirements that the user has stated, if any, and the other listing requirements the user wants to remove from the requirements list. 

For example, if the user says "I want a job in New York, with a salary of 100k", you should return the following json object:

{{
  "new_requirements": {{
    "location": "New York",
    "salary": 100000
  }},
  "remove_requirements": {{
  }}
}}

For example, if the user says "I want a job in New York, with a salary of 100k, and I don't want a job in California", you should return the following json object:

{{
  "new_requirements": {{
    "location": "New York",
    "salary": 100000
  }},
  "remove_requirements": {{
    "location": "California"
  }}
}}

"""

basePrompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            captureRequirementsPrompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = basePrompt | model


In [73]:
#| export
config = {"configurable": {"session_id": "abc6"}}

In [78]:
#| export 
def getRequirementsResponse(input:str):
    response = chain.invoke(input=[HumanMessage(input)])

    return response.content


'{\n  "new_requirements": {\n    "location": "New York",\n    "salary": 100000\n  },\n  "remove_requirements": {\n  }\n}'

In [60]:
#| export

requirements = {
    'want': {},
    'donotwant': {} 
}



In [61]:
#| export
from pydantic import BaseModel

class requirementResponse(BaseModel):
    new_requirements: dict
    remove_requirements: dict


In [62]:
#| export
# TODO: dyanamically parse response into requirement keys

def handleResponse(response: requirementResponse):
    if response.new_requirements:
        for key, value in response.new_requirements.items():
            requirements['want'][key] = value

    if response.remove_requirements:
        for key, value in response.remove_requirements.items():
            requirements['donotwant'][key] = value

            if key in requirements['want']:
                del requirements['want'][key]

    return response

In [63]:
#| export
from langchain_core.prompts import ChatPromptTemplate

def getRequirements():
    return requirements

requirementsPrompt = ChatPromptTemplate.from_template(
"""Current search requirements are: {requirements}""")

def getRequirementsPrompt():
    return requirementsPrompt.format(requirements=requirements)

getRequirementsPrompt()


"Human: Current search requirements are: {'want': {}, 'donotwant': {}}"

In [64]:
#| export
import json
def handleSubmit(input:str):
    print('input', input)
    response = getRequirementsResponse(input)
    print('response', response)
    jsonResponse = json.loads(response)
    print('jsonResponse', jsonResponse)
    validated_response = requirementResponse(**jsonResponse)

    handleResponse(validated_response)

    return getRequirementsPrompt()

In [65]:
import ipywidgets as widgets
from IPython.display import display
import json

# Create input widgets
text_input = widgets.Text()

# Create a button
submit_button = widgets.Button(description='Submit')

# Define a function to handle form submission
def handle_submit(button):
   handleSubmit(text_input.value)

# Attach the function to the button's on_click event
submit_button.on_click(handle_submit)

# Display the form
display(text_input, submit_button)

Text(value='')

Button(description='Submit', style=ButtonStyle())

input I want.
input: I want.


ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [80]:
#| hide
from nbdev import nbdev_export
nbdev_export()